Crear un sistema de procesamiento en real time de tweets en castellano usando Python y Pyspark.

 

-***(CALLER) Crear un programa Python que se conectevía socket a la API de Twitter (https://stream.twitter.com) y que descargue enreal time datos sobre algunas # de interés en castellano, enviar los datosrecibidos al Spark Streamming Server (2 puntos)***

**(SERVER) Crear una aplicación Spark StreammingServer para recibir y procesar los datos recibidos en tiempo real (2 puntos)**

***(SQL) Guardar los datos sumarizados de SparkStreamming en una base de dados SQL, ejemplo Sqilite (1 punto)***

***Traducir del castellano al inglés usando alguna API de Python vía Spark en el SERVER con los Tweets recibidos del CALLER (1punto)***

Aplicar análisis de sentimiento como vaderSentiment de Python via Spark en el SERVER con los Tweets recibidos del CALLER (1 punto)

Aplicar algún modelo de Machine Learning a los datos (hasta 3 puntos, porque elimina la necesidad de Vader - apartado 5 - y imagino que lo hariais en castellano en ML, por lo que también elimina la necesidad del apartado 4)

Calidad de la Presentación y Exposión, enseñar el sistema ejecutando en directo / valoraré si se ha hecho un dashboard para la presentación de los datos del SQL (2 puntos)


Ejemplo vaderSentiment:

vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

polarity_scores("The only reason the U.S. has reported onemillion cases of CoronaVirus is that our Testing is sooo much better than anyother country in the World. Other countries are way behind us in Testing, andtherefore show far fewer cases!")

In [52]:
!pip install google_trans_new

----------- 2021-05-16 13:16:08 -----------
----------- 2021-05-16 13:16:10 -----------
----------- 2021-05-16 13:16:12 -----------
----------- 2021-05-16 13:16:14 -----------


In [1]:
import findspark
findspark.init()
import pyspark
import traceback
import json
from pyspark.sql.session import SparkSession
import os
from google_trans_new import google_translator  
import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

conf = pyspark.SparkConf()
conf.set("spark.executor.extraClassPath",  os.path.join(os.getcwd(), 'extras/sqlite-jdbc-3.34.0.jar'))
conf.set("spark.driver.extraClassPath", os.path.join(os.getcwd(), 'extras/sqlite-jdbc-3.34.0.jar'))

sc = pyspark.SparkContext(appName="TwitterStreamApp", conf=conf)

spark = SparkSession(sc)
translator = google_translator()  

In [33]:
url = 'jdbc:sqlite:' + os.path.join(os.getcwd(), 'database.sql')

df = spark.read.jdbc(url, 'Tweet')
df.createOrReplaceTempView('Tweet')

In [3]:
!export JAVA_HOME='/usr/lib/jvm/java-8-openjdk-amd64'
!export PATH=$JAVA_HOME/bin:$PATH

In [54]:
def sentiment_score(tweet):
  
    sid = SentimentIntensityAnalyzer()
  
    sentiment = sid.polarity_scores(tweet)
      
    if sentiment['compound'] >= 0.05:
        return "Positive"
    elif sentiment['compound'] <= - 0.05:
        return "Negative"
    else:
        return 'Neutral'

In [47]:
#def aggregate_tags_count(new_values, total_sum):
    #return sum(new_values) + (total_sum or 0)
def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']

def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
        # obtén el contexto spark sql singleton desde el contexto actual
        sql_context = get_sql_context_instance(rdd.context)
        # convierte el RDD a Row RDD
        row_rdd = rdd.map(lambda x: (x, translator.translate(
                x['extended_tweet']['full_text'] if x['truncated'] else x['text'], lang_src='es', lang_tgt='en'
            ))).map(lambda x: Row(
            name=x[0]['user']['screen_name'],
            date=datetime.datetime.fromtimestamp(int(x[0]['timestamp_ms']) / 1000).strftime('%Y-%m-%d %H:%M:%S'),
            text=x[0]['extended_tweet']['full_text'] if x[0]['truncated'] else x[0]['text'],
            transtext=x[1],
            sentiment=sentiment_score(x[1])
        ))
        #row_rdd = rdd.map(lambda w: Row(hashtag=w[0], hashtag_count=w[1]))
        # crea un DF desde el Row RDD
        tweets_df = sql_context.createDataFrame(row_rdd)
        tweets_df.write.mode("append").jdbc(url, 'Tweet')
        # Registra el marco de data como tabla
        #hashtags_df.registerTempTable("hashtags")
        
        # obtén los 10 mejores hashtags de la tabla utilizando SQL e imprímelos
        #hashtag_counts_df = sql_context.sql("select hashtag, hashtag_count from hashtags order by hashtag_count desc limit 10")
        #hashtag_counts_df.show()
        
        # llama a este método para preparar los 10 mejores hashtags DF y envíalos
        #print("seve")
        #send_df_to_dashboard(hashtag_counts_df)
    except:
        #traceback.print_exception(type(ex), ex, ex.__traceback__)
        pass

----------- 2021-05-16 13:15:12 -----------
----------- 2021-05-16 13:15:14 -----------
----------- 2021-05-16 13:15:16 -----------
----------- 2021-05-16 13:15:18 -----------
----------- 2021-05-16 13:15:20 -----------
----------- 2021-05-16 13:15:22 -----------


In [5]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests

In [ ]:

# crea un contexto spark con la configuración anterior
sc.setLogLevel("ERROR")
# crea el Contexto Streaming desde el contexto spark visto arriba con intervalo de 2 segundos
ssc = StreamingContext(sc, 2)
# establece un punto de control para permitir la recuperación de RDD
ssc.checkpoint("checkpoint_TwitterApp")
# lee data del puerto 9009
dataStream = ssc.socketTextStream("localhost",9009)
# divide cada Tweet en palabras
#words = dataStream.flatMap(lambda line: line.split(" ")).map(lambda x: (x, 1))
tweets = dataStream.map(lambda x: json.loads(x))
#words.pprint()
#words.foreachRDD(process_rdd)

# agrega la cuenta de cada hashtag a su última cuenta
#words_totals = words.updateStateByKey(aggregate_tags_count)
# procesa cada RDD generado en cada intervalo
tweets.foreachRDD(process_rdd)
mySsc = ssc

# comienza la computación de streaming
ssc.start()
# espera que la transmisión termine
ssc.awaitTermination()


----------- 2021-05-16 13:19:14 -----------
----------- 2021-05-16 13:19:16 -----------
----------- 2021-05-16 13:19:18 -----------
----------- 2021-05-16 13:19:20 -----------
----------- 2021-05-16 13:19:22 -----------
----------- 2021-05-16 13:19:24 -----------
----------- 2021-05-16 13:19:26 -----------
----------- 2021-05-16 13:19:28 -----------
----------- 2021-05-16 13:19:30 -----------
----------- 2021-05-16 13:19:32 -----------
----------- 2021-05-16 13:19:34 -----------
----------- 2021-05-16 13:19:36 -----------
----------- 2021-05-16 13:19:38 -----------
----------- 2021-05-16 13:19:40 -----------
----------- 2021-05-16 13:19:42 -----------
----------- 2021-05-16 13:19:44 -----------
----------- 2021-05-16 13:19:46 -----------
----------- 2021-05-16 13:19:48 -----------
----------- 2021-05-16 13:19:50 -----------
----------- 2021-05-16 13:19:52 -----------
----------- 2021-05-16 13:19:54 -----------
----------- 2021-05-16 13:19:56 -----------
----------- 2021-05-16 13:19:58 

In [58]:
ssc.stop(False)